# Assignment 2: Support Vector Machines - SVR

Hi all, 

Please use google to find out SVM python code and then use it to further produce prediction results (regression and classification). 

With warm regards,

Stanley

## 支持向量回歸（SVR）

SVR是一種用於回歸任務的支持向量機。其目的是找到一個最佳的函數來預測連續型變量。SVR的目標是找到一個函數，使得預測值與實際值之間的誤差在某個容忍範圍內（通常稱為epsilon-insensitive loss）最小化。

### 主要特點：
- **回歸任務**：SVR適用於預測連續型變量。
- **epsilon-insensitive loss**：允許預測值與實際值之間存在一定的誤差（epsilon），在這個範圍內的誤差不會對模型的損失函數產生影響。
- **支持向量**：決定回歸函數形狀的數據點。
- **核函數**：同樣可以使用不同的核函數來處理線性和非線性回歸問題。

In [ ]:
from sklearn.svm import SVC, SVR

# SVC 示例
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# SVR 示例
reg = SVR(kernel='rbf')
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
